In [1]:
# import library
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

## Add Path

In [ ]:
OUTPUT_RELATIVE_PATH = '../../data/'

def create_path():
    """
     This function aims to create path
     """
    path = ["curated/ors-data", "raw/ors-data",
            'curated/model-data', 'raw/model-data']
    for target_dir in path:
        if not os.path.exists(OUTPUT_RELATIVE_PATH + target_dir):
            os.makedirs(OUTPUT_RELATIVE_PATH + target_dir)
        print('Already Create Paths')
    
create_path()

Already Create Paths
Already Create Paths
Already Create Paths
Already Create Paths


## Download all preprocessing Data

In [ ]:
# change directory
import os

os.chdir('../../scripts/preprocessing/')

In [ ]:
# download pre-external data
from domain_preprocessing import *

domain_preprocessing()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/30 23:03:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/30 23:03:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


()

In [ ]:
from external_preprocessing import *

preprocessing.preprocessing_all()

In [ ]:
from external_preprocessing import *

school_output()

## Join all external features

In [ ]:
from join import *

external_features()

## External-features-preprocessing

In [2]:
# read data
external_features = pd.read_csv('../../data/curated/external-data/external-feature.csv')
internal_features = pd.read_csv('../../data/curated/domain-website-data/domain_preprocessed.csv')

In [3]:
external_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   postcode            180 non-null    int64  
 1   public_duration     130 non-null    float64
 2   care_duration       180 non-null    float64
 3   shopping_duration   88 non-null     float64
 4   station_duration    107 non-null    float64
 5   hospital_duration   95 non-null     float64
 6   CBD_duration        180 non-null    float64
 7   1-10_x              72 non-null     float64
 8   11-50_x             72 non-null     float64
 9   51-100_x            72 non-null     float64
 10  101-150_x           72 non-null     float64
 11  150+_x              72 non-null     float64
 12  1-10_y              155 non-null    float64
 13  11-50_y             155 non-null    float64
 14  51-100_y            155 non-null    float64
 15  101-150_y           155 non-null    float64
 16  150+_y  

### school preprocessing
    - assumption: NaN value means there is no school wirh rank at this level
    - fill 0 with NaN value

In [4]:
pri_school_features = ['1-10_x', '11-50_x', '51-100_x', '101-150_x', '150+_x']
sec_school_features = ['1-10_y', '11-50_y', '51-100_y', '101-150_y', '150+_y']

for col in sec_school_features + pri_school_features:
    external_features[col].fillna(value = 0, inplace=True)

### crime rate preprocessing
    - fill median with NaN value

In [5]:
external_features['crime_rate'] = external_features['crime_rate'].fillna(external_features['crime_rate'].median())

### duration preprocessing
    - assumption: if the postcode have null value in these duration feature represent a long distance between the property and those facilities
    - covert the unit of durations from seconds to minutes
    - fill max value with NaN value

In [6]:
# convert seconds to minutes
def sec_to_min(lst_of_col, df):
    for col in lst_of_col:
        df[col] = df[col] / 60
    return

durations = ['public_duration', 'care_duration', 'shopping_duration', 'station_duration', 'hospital_duration', 
            'CBD_duration', 'emergency_duration']
sec_to_min(durations, external_features)

In [7]:
# fill with max value
# categorise the duration features
def fill_in_duration(lst_col, df):
    for col in lst_col:
        df[col] = df[col].fillna(max(df[col]))
    return

fill_in_duration(durations, external_features)

In [8]:
# rename columns
column_names_rename_external = ['postcode', 'public-service-duration', 'care-facility-duration', 'shopping-center-duration',
                                'train-station-duration', 'hospital-duration', 'CBD-duration', 'pri-1-10', 
                                'pri-11-50', 'pri-51-100', 'pri-101-150', 'pri-150+',
                                'sec-1-10', 'sec-11-50', 'sec-51-100', 'sec-101-150', 
                                'sec-150+', 'property-count', 'elector-count', 'emergency-service-duration',
                                'crime-rate', 'income']

external_features.set_axis(column_names_rename_external, axis = 'columns', inplace=True)

In [9]:
# selected new columns
column_names_select_external = ['postcode', 'public-service-duration', 'care-facility-duration', 'shopping-center-duration',
                                'train-station-duration', 'hospital-duration', 'CBD-duration', 'emergency-service-duration', 
                                'property-count', 'elector-count', 'crime-rate', 'income',
                                'pri-1-10', 'pri-11-50', 'pri-51-100', 'pri-101-150', 
                                'pri-150+', 'sec-1-10', 'sec-11-50', 'sec-51-100', 
                                'sec-101-150', 'sec-150+']

external_features_preprocessed = external_features[column_names_select_external]
external_features_preprocessed

,postcode,public-service-duration,care-facility-duration,shopping-center-duration,train-station-duration,hospital-duration,CBD-duration,emergency-service-duration,property-count,elector-count,...,pri-1-10,pri-11-50,pri-51-100,pri-101-150,pri-150+,sec-1-10,sec-11-50,sec-51-100,sec-101-150,sec-150+
0,3131,4.307833,1.057667,4.677667,3.092833,4.126667,27.310000,3.160000,9685.0,14758.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,3939,14.985667,5.437000,17.826667,16.598000,12.178333,79.431667,10.295000,8923.0,12193.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,3429,6.806667,4.383833,6.282667,5.422667,9.713333,44.228333,6.213333,15253.0,27933.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
3,3144,1.856500,0.697500,4.814500,2.401500,1.378333,16.446667,1.726667,5276.0,8317.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0
4,3338,9.357833,2.178833,17.826667,16.598000,18.726667,42.541667,6.443333,11175.0,18240.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,3003,14.985667,4.907500,17.826667,5.792333,18.726667,8.316667,6.166667,3068.0,3265.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0
176,3008,1.458333,1.605667,3.453500,3.836000,3.426667,5.325000,3.546667,6312.0,5355.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
177,3207,5.711667,2.173167,17.826667,16.598000,18.726667,10.823333,2.526667,9251.0,12477.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
178,3032,4.016833,3.021667,6.206667,6.085667,3.236667,16.068333,16.523333,13659.0,20396.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


## Internal-features-preprocessing

Specification
  - Price
  - Bedrooms
  - Bathrooms
  - Parking
  - Property_type
  - Postcode_property_count
  - coordinates

In [10]:
internal_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12127 entries, 0 to 12126
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               12127 non-null  int64  
 1   property_id              12127 non-null  int64  
 2   street                   12127 non-null  object 
 3   suburb                   12127 non-null  object 
 4   postcode                 12127 non-null  int64  
 5   latitude                 12127 non-null  float64
 6   longitude                12127 non-null  float64
 7   price                    12127 non-null  object 
 8   bedrooms                 12127 non-null  int64  
 9   bathrooms                12127 non-null  int64  
 10  parking                  12127 non-null  int64  
 11  property_type            12127 non-null  object 
 12  url                      12127 non-null  object 
 13  price_rent               12127 non-null  float64
 14  coordinate            

In [11]:
# selected columns
column_names_select_internal = ['postcode', 'street', 'suburb', 'latitude', 
                                'longitude', 'bedrooms', 'bathrooms', 'parking', 
                                'property_type', 'price_rent', 'postcode_property_count']

internal_features = internal_features[column_names_select_internal]

In [12]:
# rename columns
column_names_rename_internal = ['postcode', 'street', 'suburb', 'latitude', 
                                'longitude', 'bedrooms', 'bathrooms', 'parking',
                                'property_type', 'price', 'postcode_property_count']

internal_features_preprocessed = internal_features.set_axis(column_names_rename_internal, axis = 'columns')
internal_features_preprocessed

,postcode,street,suburb,latitude,longitude,bedrooms,bathrooms,parking,property_type,price,postcode_property_count
0,3029,33 cindia crescent,TARNEIT,-37.836800,144.681992,3,2,2,House,410.0,264
1,3029,3 loire close,HOPPERS CROSSING,-37.858776,144.670990,4,2,2,House,420.0,264
2,3029,5 toscana,TRUGANINA,-37.831726,144.721200,4,2,2,New House & Land,440.0,264
3,3029,12 sussex court,TARNEIT,-37.850758,144.712234,3,3,1,House,415.0,264
4,3029,4a craig close,TRUGANINA,-37.843980,144.717514,3,2,1,House,410.0,264
...,...,...,...,...,...,...,...,...,...,...,...
12122,3934,15/26 green island avenue,MOUNT MARTHA,-38.248100,145.041611,3,2,2,Townhouse,625.0,13
12123,3934,8 raymond street,MOUNT MARTHA,-38.285440,144.995331,4,2,2,House,720.0,13
12124,3934,15 whitsunday ct,MOUNT MARTHA,-38.250070,145.052078,4,2,2,House,700.0,13
12125,3934,107 forest dr,MOUNT MARTHA,-38.290226,145.016418,3,2,2,House,870.0,13


## Join external and internal features

In [13]:
# join internal features with external features
features_df = pd.merge(internal_features_preprocessed, external_features_preprocessed, on='postcode', how='outer')

## Merge with population

In [14]:
df_population_growth_rate = pd.read_csv("../../data/raw/external-data/population_growth.csv")

# read shapefile
df_shape = pd.DataFrame(gpd.read_file("../../data/external-raw-data/SA2_2021_AUST_SHP_GDA2020 2/SA2_2021_AUST_GDA2020.shp"))

In [15]:
df_shape = df_shape.dropna().reset_index()
features_df["SA2_code"] = ""

# add SA2 region code for each property
for i in range(len(features_df)):
    property_location = Point(features_df['longitude'].iloc[i], features_df['latitude'].iloc[i])
    code = df_shape.loc[df_shape["geometry"].apply(lambda x: x.contains(property_location))]["SA2_CODE21"]
    code = int(code.values[0])
    features_df.loc[i, "SA2_code"]=code

In [16]:
# join growth rate and external feature data by SA2 code
df = features_df.merge(df_population_growth_rate, left_on='SA2_code', right_on='SA2', how='left')

In [17]:
# drop usedless columns
df = df.drop(['Unnamed: 0', 'SA2', 'Area Name','SA2_code'], axis=1)

In [18]:
# fill null value with median of population growth rate
df['population_growth_rate_2021-2031'] = df['population_growth_rate_2021-2031'].fillna(df['population_growth_rate_2021-2031'].median())

In [19]:
df['street'] = df['street'].astype("string")
df['suburb'] = df['suburb'].astype("string")
df['property_type'] = df['property_type'].astype("string")

In [26]:
df = df.rename({'population_growth_rate_2021-2031' : 'population-growth-rate'}, axis=1)

In [28]:
df.to_csv("../../data/curated/model-data/model-data.csv")